Jisaqで使用しているパッケージ
- `TensorKit`
- `BenchmarkTools`
- `KrylovKit`

加えて、このnotebookで使用しているパッケージ
- `Plots`
- `LaTeXStrings`

In [ ]:
include("Jisaq.jl")

In [ ]:
using .Jisaq, Plots, LaTeXStrings

In [ ]:
nq = 10
dt = 0.1
each_Rx = Circuit([Rx(i, -dt) for i in 1:nq])
each_Rzz = Circuit([Rzz(i, i+1, -dt) for i in 1:nq-1])
step = Circuit([each_Rx, each_Rzz])
draw(step, :unicode)

In [ ]:
sv = Statevector(nq)
bond = 4
mps = OpenMPS(nq, bond)
n = 50
obs = Z(1)
expect_sv = Vector{Float64}(undef, n+1)
expect_sv[1] = real(expect(sv, obs))
expect_mps = Vector{Float64}(undef, n+1)
expect_mps[1] = real(expect(mps, obs))
time = Vector{Float64}(undef, n+1)
time[1] = 0.0

for i in 2:n+1
    apply!(sv, step)
    apply!(mps, step)
    time[i] = dt * (i - 1)
    expect_sv[i] = real(expect(sv, obs))
    expect_mps[i] = real(expect(mps, obs))
end

In [ ]:
plot(;xlabel=L"t", ylabel=L"\langle Z_1 \rangle")
plot!(time, expect_sv; label="statevector simulator")
plot!(time, expect_mps; label="mps simulator (bond=$bond)")